In [ ]:
from __future__ import annotations
import polars as pl 
import inspect

In [ ]:
test = ["a","b"]

"|".join(test)

In [ ]:
def test() -> list[str]:
    return ["a"]

test.__annotations__.get("return", "") == list[str]

In [ ]:
from typing import Callable, Concatenate

tt:Callable[[], list]
tt = test

In [ ]:
inspect.signature(tt).return_annotation == "list[str]"

In [ ]:
df = pl.DataFrame({
    "a":[1,1,1,1]
    , "b":[2,2,2,2]
    , "c":[3,3,3,3]
})

test = pl.Series("x",["a", "b"])


df.select(["a", "b"])

In [ ]:
df = pl.DataFrame(
    {
        "a": ["a", "b", "a", "b", "c"],
        "b": [1, 2, 1, 3, 3],
        "c": [5, 4, 3, 2, 1],
    }
)

In [ ]:
for f in df.partition_by("a"):
    print(f.shape)

In [ ]:
df.select(
    pl.col(c).n_unique() for c in df.columns
)

In [ ]:
d = pl.DataFrame(
    {
        "id": ["a", "b", "c", "d"],
        "events": [["e1", "e2"], ["e1", "e2"], ["e1", "e3"], [None]],
    }
)

In [ ]:
d.select(
    pl.col("events").list.unique().list.lengths()
)

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
orig_x, orig_y = make_classification(n_samples = 300_000, n_features = 50, n_informative = 25, n_redundant = 25)
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))

In [ ]:
features = df.columns
features.remove("target")

In [ ]:
x = df.select(features)
v = pl.Series([2]*len(features))

print(x.shape)
print(v.shape)

In [ ]:
%%timeit
x.select(
    pl.col(c) * v[i] for i, c in enumerate(x.columns)
).fold(lambda s1, s2: s1 + s2)

In [ ]:
%%timeit
x.select(
    sum(pl.col(c) * v[i] for i, c in enumerate(x.columns))
)

In [ ]:
w = pl.Series([2]*200)


In [ ]:
def gradient_descent(df:pl.DataFrame, features:list[str], target:str):
    
    x = df.select(features)
    y = df.select(target)

    weights = pl.Series([1]*len(features))
    const = 0.

    


In [1]:
import polars as pl
from dsds.prescreen import describe_str

In [2]:
df = pl.read_csv("../data/advertising.csv")

In [3]:
describe_str(df, words_to_count=["A"])

features,null_count,null_pct,min,max,mode,min_byte_len,max_byte_len,avg_byte_len,median_byte_len,avg_space_cnt,avg_digit_cnt,avg_cap_cnt,avg_lower_cnt,total_A_count
str,u32,f64,str,str,str,u32,u32,f64,f64,f64,f64,f64,f64,u32
"""Ad Topic Line""",0,0.0,"""Adaptive 24hou…","""Visionary reci…","""Exclusive clie…",17,55,33.394,33.0,2.219,0.087,1.108,29.221,60
"""City""",0,0.0,"""Adamsbury""","""Zacharyton""","""Williamsport""",5,23,11.887,11.0,0.471,0.0,1.471,9.945,65
"""Country""",0,0.0,"""Afghanistan""","""Zimbabwe""","""France""",4,51,10.309,8.0,0.494,0.006,1.424,8.328,107
"""Timestamp""",0,0.0,"""1/1/2016 15:14…","""7/9/2016 16:23…","""5/20/2016 12:1…",13,15,14.273,14.0,1.0,10.273,0.0,0.0,0
"""One_Hot_Test""",0,0.0,"""A""","""C""","""A""",1,1,1.0,1.0,0.0,0.0,1.0,0.0,783
